<a href="https://colab.research.google.com/github/sanha-hwang/ocular_disease/blob/main/ocular_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Project4_level1: 목표

1. 캐글에서 안구질환 인식 데이터를 받아 안구질환을 진단하는 모델을 만들어보자

2. 5000명의 환자의 안저 사진 데이터가 있고 라벨로는
일반 : N, 당뇨병 : D, 녹내장 : G, 백내장 : C, 연령 관련 황반변성 : A
고혈압 : H, 병적 근시 : M, 기타 질병 / 이상 : O

Normal (N),
Diabetes (D),
Glaucoma (G),
Cataract (C),
Age related Macular Degeneration (A),
Hypertension (H),
Pathological Myopia (M),
Other diseases/abnormalities (O)

3. 이미지 분류 잘하는 모델 만들기 목표 80프로 이상

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow
from tensorflow import keras


In [4]:
base_dir = '/content/drive/MyDrive/Input/ocular-disease-recognition-odir-5k/ODIR-5K'

train_dir = os.path.join(base_dir,'Training Images')
dataset_path = os.path.join(base_dir,'data.xlsx')

# train_fname = os.listdir(train_dir)

In [5]:
dataset = pd.read_excel(dataset_path)
dataset.head(10)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
5,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0
6,6,60,Male,6_left.jpg,6_right.jpg,macular epiretinal membrane,moderate non proliferative retinopathy，epireti...,0,1,0,0,0,0,0,1
7,7,60,Female,7_left.jpg,7_right.jpg,drusen,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1
8,8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
9,9,54,Male,9_left.jpg,9_right.jpg,normal fundus,vitreous degeneration,0,0,0,0,0,0,0,1


In [28]:
dataset.columns

Index(['ID', 'Patient Age', 'Patient Sex', 'Left-Fundus', 'Right-Fundus',
       'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords', 'N', 'D', 'G',
       'C', 'A', 'H', 'M', 'O'],
      dtype='object')

In [30]:
patient_info = dataset.loc[:,['ID','Patient Age','Patient Sex']]

In [ ]:
patient_info.head()

In [32]:
# male : 0, female : 1
def gender(text):
  text = text.lower()
  if 'male' in text:
    return 0
  elif 'female' in text:
    return 1

patient_info['Patient Sex'] = patient_info['Patient Sex'].apply(gender)

In [131]:
left_info = dataset.loc[:,['Left-Fundus','Left-Diagnostic Keywords']]
right_info = dataset.loc[:,['Right-Fundus','Right-Diagnostic Keywords']]                      

In [133]:
def label(data, where):
  condition = where+'-Diagnostic Keywords'
  data['N'] = data[condition].str.contains('normal').apply(int)
  data['D'] = data[condition].str.contains('proliferative|diabe').apply(int)
  data['G'] = data[condition].str.contains('glaucoma').apply(int)
  data['C'] = data[condition].str.contains('cataract').apply(int)
  data['A'] = data[condition].str.contains('age').apply(int)
  data['H'] = data[condition].str.contains('hyper').apply(int)
  data['M'] = data[condition].str.contains('myopia').apply(int)

  for i in range(len(data)):
    if data.loc[i,['N','D','G','C','A','H','M']].sum() ==0:
      data.loc[i,['O']] = True
    else:
      data.loc[i,['O']] = False 

  data['O'] = data['O'].apply(int)

label(left_info,'Left')
label(right_info, 'Right')

In [2]:
from keras.models import load_model

# model.save('mnist_mlp_model.h5')

In [ ]:
import random
image_size=224
dataset = []

for img in left_info['Left-Fundus'].values:
  image_path = os.path.join(train_dir,img)
  try:
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(image_size,image_size))
  except:
    continue
  condition = left_info['Left-Fundus'] == img
  index = left_info[condition].index[0]
  label = [left_info.loc[index,['N']].values[0],left_info.loc[index,['D']].values[0],left_info.loc[index,['G']].values[0],
                             left_info.loc[index,['C']].values[0],left_info.loc[index,['A']].values[0], left_info.loc[index,['H']].values[0],
                             left_info.loc[index,['M']].values[0], left_info.loc[index,['O']].values[0]]
  dataset.append([np.array(image),np.array(label)])

for img in right_info['Right-Fundus'].values:
  image_path = os.path.join(train_dir,img)
  try:
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(image_size,image_size))
  except:
    continue
  condition = right_info['Rightt-Fundus'] == img
  index = right_info[condition].index[0]
  label = [right_info.loc[index,['N']].values[0],right_info.loc[index,['D']].values[0],right_info.loc[index,['G']].values[0],
                             right_info.loc[index,['C']].values[0],right_info.loc[index,['A']].values[0], right_info.loc[index,['H']].values[0],
                             right_info.loc[index,['M']].values[0], right_info.loc[index,['O']].values[0]]
  dataset.append([np.array(image),np.array(label)])

random.shuffle(dataset)


from sklearn.model_selection import train_test_split

x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)

history = model.fit(x_train,y_train,batch_size=32,epochs=1,validation_data=(x_test,y_test),
                    verbose=1) #,callbacks=[checkpoint,earlystop])



/usr/local/lib/python3.7/dist-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from keras.models import load_model
import tensorflow
from tensorflow import keras
from tensorflow.keras import optimizers
import random
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

#사용자정의함수
def gender(text):
  text = text.lower()
  if 'male' in text:
    return 0
  elif 'female' in text:
    return 1

def label(data, where):
  condition = where+'-Diagnostic Keywords'
  data['N'] = data[condition].str.contains('normal').apply(int)
  data['D'] = data[condition].str.contains('proliferative|diabe').apply(int)
  data['G'] = data[condition].str.contains('glaucoma').apply(int)
  data['C'] = data[condition].str.contains('cataract').apply(int)
  data['A'] = data[condition].str.contains('age').apply(int)
  data['H'] = data[condition].str.contains('hyper').apply(int)
  data['M'] = data[condition].str.contains('myopia').apply(int)

  for i in range(len(data)):
    if data.loc[i,['N','D','G','C','A','H','M']].sum() ==0:
      data.loc[i,['O']] = True
    else:
      data.loc[i,['O']] = False 

  data['O'] = data['O'].apply(int)

#데이터 불러오기
base_dir = '/content/drive/MyDrive/Input/ocular-disease-recognition-odir-5k/ODIR-5K'

train_dir = os.path.join(base_dir,'Training Images')
dataset_path = os.path.join(base_dir,'data.xlsx')
dataset = pd.read_excel(dataset_path)

#데이터전처리
patient_info = dataset.loc[:,['ID','Patient Age','Patient Sex']]
patient_info['Patient Sex'] = patient_info['Patient Sex'].apply(gender)

left_info = dataset.loc[:,['Left-Fundus','Left-Diagnostic Keywords']]
right_info = dataset.loc[:,['Right-Fundus','Right-Diagnostic Keywords']]
label(left_info,'Left')
label(right_info, 'Right')

#데이터 모으기
image_size=224
dataset = []

for img in left_info['Left-Fundus'].values:
  image_path = os.path.join(train_dir,img)
  try:
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(image_size,image_size))
  except:
    continue
  condition = left_info['Left-Fundus'] == img
  index = left_info[condition].index[0]
  label = [left_info.loc[index,['N']].values[0],left_info.loc[index,['D']].values[0],left_info.loc[index,['G']].values[0],
                             left_info.loc[index,['C']].values[0],left_info.loc[index,['A']].values[0], left_info.loc[index,['H']].values[0],
                             left_info.loc[index,['M']].values[0], left_info.loc[index,['O']].values[0]]
  dataset.append([np.array(image),np.array(label)])

for img in right_info['Right-Fundus'].values:
  image_path = os.path.join(train_dir,img)
  try:
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(image_size,image_size))
  except:
    continue
  condition = right_info['Rightt-Fundus'] == img
  index = right_info[condition].index[0]
  label = [right_info.loc[index,['N']].values[0],right_info.loc[index,['D']].values[0],right_info.loc[index,['G']].values[0],
                             right_info.loc[index,['C']].values[0],right_info.loc[index,['A']].values[0], right_info.loc[index,['H']].values[0],
                             right_info.loc[index,['M']].values[0], right_info.loc[index,['O']].values[0]]
  dataset.append([np.array(image),np.array(label)])

random.shuffle(dataset)

#모델학습에 넣기 좋게 바꾸기
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)


#모델정의 변수들

Num_classes = 8 # N, D, G, C, A, H, M, O

channels = 3 # 입력 이미지의 차원수 : RBG

  #학습된 네트워크 Resnet50
image_size =224
resnet_pooling_average = 'avg'
dense_layer_activation = 'softmax'
objective_function = 'categorical_crossentropy'

  #출력 metrics
loss_metrics = ['accuracy']

  #early_stop
num_epochs = 10
early_stop_patience = 3

step_per_epoch_training = 10
step_per_epoch_validation = 10 

batch_size_training = 100
batch_size_validation = 100

batch_size_testing = 1

#모델 작성

model = Sequential()
model.add(ResNet50(include_top = False, pooling = resnet_pooling_average))
model.add(Dense(Num_classes, activation = dense_layer_activation))

for layer in model.layers:
  layer.trainable = False

#모델 컴파일러 정의

model.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)

#모델학습
history = model.fit(x_train,y_train,batch_size=32,epochs=1,validation_data=(x_test,y_test),
                    verbose=1) #,callbacks=[checkpoint,earlystop])

#모델저장

model.save('samplemodel.h5')

In [1]:
#Step. model define 

from keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

Num_classes = 8 # N, D, G, C, A, H, M, O

channels = 3 # 입력 이미지의 차원수 : RBG

#학습된 네트워크 Resnet50
image_size =224
resnet_pooling_average = 'avg'
dense_layer_activation = 'softmax'
objective_function = 'categorical_crossentropy'

#출력 metrics
loss_metrics = ['accuracy']

#early_stop
num_epochs = 10
early_stop_patience = 3

step_per_epoch_training = 10
step_per_epoch_validation = 10 

batch_size_training = 100
batch_size_validation = 100

batch_size_testing = 1

#Step model making

model = Sequential()
model.add(ResNet50(include_top = False, pooling = resnet_pooling_average))
model.add(Dense(Num_classes, activation = dense_layer_activation))

for layer in model.layers:
  layer.trainable = False

#Step model compiler

from tensorflow.keras import optimizers

model.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)

 

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
#try
from sklearn.model_selection import train_test_split

x = np.array([i[0] for i in datasets]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in datasets])

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)

history = model.fit(x_train,y_train,batch_size=32,epochs=1,validation_data=(x_test,y_test),
                    verbose=1) #,callbacks=[checkpoint,earlystop])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 0
Non-trainable params: 23,604,104
_________________________________________________________________


In [ ]:
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

In [ ]:
from sklearn.model_selection import train_test_split

train_data, validation_data = train_test_split(dataset, test_size= 0.2, random_state= 42)

train_data.shape, validation_data.shape

((2800, 15), (700, 15))

In [ ]:
#번외편 이미지 한번 눈으로 확인해보기
%matplotlib inline

# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt

# nrows, ncols = 4, 4
# pic_index = 0

# fig = plt.gcf()
# fig.set_size_inches(ncols*3, nrows*3)

# pic_index+=8

# next_ocular_pix = [os.path.join(train_dir, fname)
#                 for fname in train_fname[pic_index-8:pic_index]]

# for i, img_path in enumerate(next_ocular_pix):
#   sp = plt.subplot(nrows, ncols, i + 1)
#   sp.axis('Off')
#   img = mpimg.imread(img_path)
#   plt.imshow(img)

# plt.show()

In [ ]:
# Step input image resize & augmentation

from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
    rescale = 1.0/255.,
    preprocessing_function = preprocess_input,
    featurewise_center = True,
    featurewise_std_normalization = True,
    rotation_state = 20,
    width_sift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True)

train_generator = data_generator.flow_from_directory(trainset_dir,
                                                     batch_size = batch_size_training,
                                                     class_mode = 'categorical')

validation_gen = ImageDataGenerator( rescale = 1.0/255.)
validation_generator = validation_gen.flow_from_directory(validation_dir,
                                                         batch_size = batch_size_training,
                                                         class_mode = 'categorical')

In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=1,validation_data=(x_test,y_test),
                    verbose=1) #,callbacks=[checkpoint,earlystop])

In [ ]:
data_generator.fit(train_images)

In [ ]:
#Step model learning
# model.fit(data_generator.flow(train_images, train_label, batch_size = 32), step_per_epoch = len(train_images)/ 32, epochs= num_epochs)